In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.pydantic import PydanticOutputParser
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
from pydantic import BaseModel, Field
from typing import List

In [26]:
class Query(BaseModel):
    id: int
    question: str
    dependencies: List[int] = Field(
        default_factory=list,
        description="""A list of sub-queries that must be completed before this task can be completed. 
        Use a sub query when anything is unknown and we might need to ask many queries to get an answer. 
        Dependencies must only be other queries."""
    )

class QueryPlan(BaseModel):
    query_graph: List[Query]

In [41]:
chat = ChatOpenAI()

# Set up a parser:
parser = PydanticOutputParser(pydantic_object=QueryPlan)

template = """Generate a query plan. This will be used for task execution.

Answer the following query: {query}

Return the following query graph format:
{format_instructions}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

# Add the output parser:
chat_prompt.output_parser = parser

# Format the prompt:
formatted_prompt = chat_prompt.format_prompt(
    query='''I want to get the results from my database. Then I want to find out what the average age of my top 10 customers is. 
    Once I have the average age, I want to send an email to John. Also I just generally want to send a welcome introduction email to Sarah, regardless of the other tasks.''',
    format_instructions=parser.get_format_instructions()
).to_messages()

# Format the LLM:
response = chat(formatted_prompt)

# Parse the result
query_plan = parser.parse(response.content)

print(query_plan.query_graph)

[Query(id=1, question='Get top 10 customers', dependencies=[]), Query(id=2, question='Calculate average age of customers', dependencies=[1]), Query(id=3, question='Send email to John', dependencies=[2]), Query(id=4, question='Send welcome email to Sarah', dependencies=[])]
